In [1]:
# General
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ML

from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier


# feature extraction
import biosppy.signals.ecg as ecg

# Custom
import sys,os
sys.path.append( '.' )
sys.path.append( '..' )
import Components.data_fetching as data_fetching
import Components.feature_extraction as feature_extraction
import Components.Two_Step_Model as Two_Step_Model

# CAREFUL:
# If you make changes to a custom module, you have to reload it, i.e rerun this cell
import importlib

importlib.reload(data_fetching)
importlib.reload(feature_extraction)
importlib.reload(Two_Step_Model)



from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets




In [2]:
X = feature_extraction.get_features(X=None,types=['heartbeats', 'timeseries', 'peaks'], verbose=True, precomputed='train')

In [3]:
# get test_data
X_test = feature_extraction.get_features(X=None,types=['heartbeats', 'timeseries', 'peaks'], verbose=True, precomputed='test')

In [ ]:
_, y = data_fetching.get_train_data()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (17979) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
pipe = Pipeline([('scaling',StandardScaler()),
                 ('classification',
                  XGBClassifier())
                ])

In [ ]:
#xgb params
n_trees = [6,7,8]
learning_rate = [0.05, 0.1, 0.15]
n_estimators = [100, 300, 500]
parameters = [{'classification__max_depth': n_tress,
                'classification__n_estimators': n_estimators,
              'classification__learning_rate': learning_rate}]


In [ ]:
# View The Best Parameters
#last on got an error
clf = GridSearchCV(pipe, parameters,cv=10,n_jobs=16,scoring='f1_micro', verbose=10)
clf.fit(X, y)
print(clf.best_params_)
print(clf.best_score_)

In [ ]:
y_pred = clf.predict(X_test)
y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
y_pred_pd.to_csv('../../Predictions/XGB_tuned_6-8.csv', index_label='id')